# Báo cáo cuối kì đợt 3 - 20/01/2026

Đề tài: **Customer Segmentation - Recommender Systerm**

- Môn học: Python cho khoa học dữ liệu
- Lớp: 23TTH
- Giảng viên bộ môn: ThS. Hà Văn Thảo

Thành viên
1. 21110100 - Trần Đăng Huy 
2. 21110062 - Huỳnh Tiểu Dung 
3. 21110453 - Trần Long Vũ 
4. 21110117 - Đinh Thủy Ngân Kiều 


# Mục lục

1. Giới thiệu bài toán
1. Dataset
2. EDA
3. Customer segmentation
4. Product refill recommendation

# 1. Giới thiệu bài toán 

**Custormer Segmentation** – Phân khúc khách hàng là quá trình phân chia khách hàng thành các nhóm dựa trên các đặc điểm khách hàng mục tiêu chung để các công ty có thể có cách phục vụ cho từng nhóm một cách hiệu quả và phù hợp. Đây là bước quan trọng, là yếu tố giúp tăng tỉ lệ chuyển đổi cho doanh nghiệp. Nếu doanh nghiệp thực hiện tốt bước này có thể giúp doanh nghiệp phân chia ngân sách đến đúng đối tượng hơn và tiết kiệm được nhiều hơn.


<img src="/home/huy/HCMUS/subject/Python_For_DS/images/RFM.png" width="101000" height="auto" alt="Ví dụ minh họa mô hình RFM"/>

Vì sao cần phải phân khúc khách hàng ?
- Tìm kiếm các khách hàng có giá trị cao
- Tối ưu hiệu quả của các chiến dịch marketing (ROI: Revenue on Investment)
- Cải thiện khả năng giữ chân (retention) và giảm thiểu rủi ro rời đi (churn)
- Cá nhân hóa trải nghiệm 
- Không phải khách hàng nào cũng mang đến cùng 1 tệp giá trị đến doanh nghiệp


**Refill item prediction**


# 2. Bộ dữ liệu

Bộ dữ liệu được sử dụng là lịch sử mua hàng trực tuyến trong vòng 6 tháng của một chuỗi siêu thị thực phẩm ngành hàng FMCG (Fast-Moving Consumer Goods) trên nền tảng online.

Data gồm các cột:

- **datetime** (datetime): thời gian giao dịch được thực hiện (GMT+0 tiêu chuẩn)
- **ol_invoice_code** (strstr): Mã giao dịch
- **product_variant_barcode** (str): Barcode sản phẩm
- **quantity_order** (float): Số lượng sản phẩm được mua
- **selling_price** (float): Giá bán sản phẩm
- **customer_code** (str): Mã khách hàng

In [ ]:
# Import libs 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
# Read data
transaction_df = pd.read_parquet("/home/huy/HCMUS/subject/Python_For_DS/data/preprocessed/transaction_data_processed.parquet")
transaction_df.head()

,datetime,customer_code,ol_invoice_code,product_variant_barcode,quantity_order,selling_price
0,2025-11-22 05:33:23+00:00,28740b13add3407ed4b4fb8b54fa3cbb9d7d63523ce9d7...,42563ad195c830746d9c475ede109b238381020d6896ab...,8934563138165,13.00,4000.0
1,2025-08-01 05:42:52+00:00,3afb4bd1f666a5ec394ad040a6e81099b9b225668bfede...,942198c346a3416ce71ddce8086f039c6fa4289164b3e1...,11404,1.00,12950.0
2,2025-12-01 08:09:25+00:00,5eed255c14aa30dd88e9eda80f0265b374be0892699854...,ed45456f27ae04e5be6ce1004bf9ae940345a481c230c5...,1101366,0.32,62200.0
3,2025-09-10 10:31:06+00:00,69c13dda6491e3d537283648b42143cb7e9c52f45dceed...,86e4e93e0720105885d30a17e9d7a9c27ace3cdf8ac10d...,8938504432107,1.00,16000.0
4,2025-08-16 10:07:38+00:00,d18cdad75bae89ac828d3765fa134bce2e81f7cc408f0c...,80c2a5da9ccfcd04acce57acf11fb37cf1b19c52aabfbe...,11434,1.00,179000.0


In [31]:
# data size
transaction_df.shape

(1502978, 6)

# 3. Preprocessing data

In [32]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1502978 entries, 0 to 1502983
Data columns (total 6 columns):
 #   Column                   Non-Null Count    Dtype              
---  ------                   --------------    -----              
 0   datetime                 1502978 non-null  datetime64[us, UTC]
 1   customer_code            1502978 non-null  object             
 2   ol_invoice_code          1502978 non-null  object             
 3   product_variant_barcode  1442137 non-null  object             
 4   quantity_order           1502978 non-null  float64            
 5   selling_price            1442137 non-null  float64            
dtypes: datetime64[us, UTC](1), float64(2), object(3)
memory usage: 80.3+ MB


In [33]:
# check missing values
print("Missing values: ",transaction_df.isnull().sum())



Missing values:  datetime                       0
customer_code                  0
ol_invoice_code                0
product_variant_barcode    60841
quantity_order                 0
selling_price              60841
dtype: int64


In [34]:
missing_values = transaction_df[(transaction_df['selling_price'].isnull()) | (transaction_df['selling_price'] < 0)]
len(missing_values)

60841

In [35]:
# Drop missing values
transaction_df = transaction_df.drop(missing_values.index)
len(transaction_df)

1442137

In [38]:
# Convert to GMT+7
transaction_df['datetime'] = transaction_df['datetime'] + pd.Timedelta(hours=7)
transaction_df.head()

,datetime,customer_code,ol_invoice_code,product_variant_barcode,quantity_order,selling_price
0,2025-11-22 19:33:23+00:00,28740b13add3407ed4b4fb8b54fa3cbb9d7d63523ce9d7...,42563ad195c830746d9c475ede109b238381020d6896ab...,8934563138165,13.00,4000.0
1,2025-08-01 19:42:52+00:00,3afb4bd1f666a5ec394ad040a6e81099b9b225668bfede...,942198c346a3416ce71ddce8086f039c6fa4289164b3e1...,11404,1.00,12950.0
2,2025-12-01 22:09:25+00:00,5eed255c14aa30dd88e9eda80f0265b374be0892699854...,ed45456f27ae04e5be6ce1004bf9ae940345a481c230c5...,1101366,0.32,62200.0
3,2025-09-11 00:31:06+00:00,69c13dda6491e3d537283648b42143cb7e9c52f45dceed...,86e4e93e0720105885d30a17e9d7a9c27ace3cdf8ac10d...,8938504432107,1.00,16000.0
4,2025-08-17 00:07:38+00:00,d18cdad75bae89ac828d3765fa134bce2e81f7cc408f0c...,80c2a5da9ccfcd04acce57acf11fb37cf1b19c52aabfbe...,11434,1.00,179000.0


# 3. Exploratory Data Analysis